## NO NEED TO RUN THE FIRST CELL IF FILES ARE NOT ZIPPED

## START FROM SECOND CELL


In [ ]:
# import torch
# print(torch.__version__)
# ! python -V
# !unzip src.zip
!unzip small_chair.zip

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
!pip install trimesh
!pip install tensorboardX
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.6.0.html

     |████████████████████████████████| 634kB 7.1MB/s 
     |████████████████████████████████| 317kB 7.1MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0.html
     |████████████████████████████████| 11.5MB 268kB/s 


In [ ]:
import torch
import torch.optim as optim
#from tensorboardX import SummaryWriter
import numpy as np
import os
import yaml
import argparse
import time, datetime
import matplotlib; matplotlib.use('Agg')
from collections import defaultdict
import shutil


import torch.optim as optim
from trainer import Trainer
from trainer import conv_occ_net
from encoder import local_pool_pn
from ref_encoder import LocalPoolPointnet
from decoder import local_decoder
from ref_decoder import LocalDecoder

from get_dataset import get_data_fields
from get_dataset import Shapes3dDataset
from get_dataset import get_inputs_field
from get_dataset import collate_remove_none
import generation
from checkpoints import CheckpointIO

 50%|█████     | 1/2 [00:03<00:03,  3.16s/it]

0.022787403


In [ ]:
def load_config(path, default_path=None):
    with open(path, 'r') as f:
        cfg_special = yaml.load(f)
    inherit_from = cfg_special.get('inherit_from')
    if inherit_from is not None:
        cfg = load_config(inherit_from, default_path)
    elif default_path is not None:
        with open(default_path, 'r') as f:
            cfg = yaml.load(f)
    else:
        cfg = dict()
    update_recursive(cfg, cfg_special)
    return cfg


def update_recursive(dict1, dict2):
    for k, v in dict2.items():
        if k not in dict1:
            dict1[k] = dict()
        if isinstance(v, dict):
            update_recursive(dict1[k], v)
        else:
            dict1[k] = v

In [ ]:
cfg=load_config("./shapenet_grid32.yaml","./default.yaml")
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
out_dir = cfg['training']['out_dir']
batch_size = cfg['training']['batch_size']
backup_every = cfg['training']['backup_every']
vis_n_outputs = cfg['generation']['vis_n_outputs']
exit_after = -1

model_selection_metric = cfg['training']['model_selection_metric']
if cfg['training']['model_selection_mode'] == 'maximize':
    model_selection_sign = 1
elif cfg['training']['model_selection_mode'] == 'minimize':
    model_selection_sign = -1
else:
    raise ValueError('model_selection_mode must be '
                     'either maximize or minimize.')

In [ ]:
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
# train_dataset = config.get_dataset('train', cfg)
# val_dataset = config.get_dataset('val', cfg, return_idx=True)
cfg1=cfg
mode="test"
method = cfg['method']
dataset_type = cfg['data']['dataset']
dataset_folder = cfg['data']['path']
categories = cfg['data']['classes']

splits = {'train': cfg1['data']['train_split'],'val': cfg['data']['val_split'],'test': cfg['data']['test_split'],}
split = splits[mode]
return_idx=False
if dataset_type == 'Shapes3D':
    fields=get_data_fields(mode,cfg1)
    inputs_field = get_inputs_field(mode, cfg1)
    if inputs_field is not None:
        fields['inputs'] = inputs_field
    if return_idx:
        fields['idx'] = data.IndexField()

train_dataset=Shapes3dDataset(dataset_folder, fields,split=split,categories=categories,cfg=cfg1)

In [ ]:
from src import data
mode="test"
method = cfg['method']
dataset_type = cfg['data']['dataset']
dataset_folder = cfg['data']['path']
categories = cfg['data']['classes']

splits = {'train': cfg['data']['train_split'],'val': cfg['data']['val_split'],'test': cfg['data']['test_split'],}
split = splits[mode]
return_idx=True
if dataset_type == 'Shapes3D':
    fields=get_data_fields(mode,cfg1)
    inputs_field = get_inputs_field(mode, cfg1)
    if inputs_field is not None:
        fields['inputs'] = inputs_field
    if return_idx:
        fields['idx'] = data.IndexField()

val_dataset=Shapes3dDataset(dataset_folder, fields,split=split,categories=categories,cfg=cfg1)

In [ ]:
test_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=1,num_workers=0, shuffle=False,
    collate_fn=collate_remove_none)
# val_loader = torch.utils.data.DataLoader(
#         val_dataset, batch_size=1, num_workers=cfg['training']['n_workers_val'], shuffle=False,
#     collate_fn=collate_remove_none)

In [ ]:
for batch_idx, batch in enumerate(train_loader):
  if batch_idx == 0:
    print(batch.keys())

In [ ]:
for batch_idx, batch in enumerate(val_loader):
  if batch_idx == 0:
    print(batch.keys())

dict_keys(['points', 'points.occ', 'points_iou', 'points_iou.occ', 'inputs', 'inputs.normals', 'idx'])


In [ ]:
# For visualizations
vis_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=1, shuffle=False,
    collate_fn=collate_remove_none)
model_counter = defaultdict(int)
data_vis_list = []

In [ ]:
iterator = iter(vis_loader)
for i in range(len(vis_loader)):
    data_vis = next(iterator)
    idx = data_vis['idx'].item()
    model_dict = val_dataset.get_model_dict(idx)
    category_id = model_dict.get('category', 'n/a')
    category_name = val_dataset.metadata[category_id].get('name', 'n/a')
    category_name = category_name.split(',')[0]
    if category_name == 'n/a':
        category_name = category_id

    c_it = model_counter[category_id]
    if c_it < vis_n_outputs:
        data_vis_list.append({'category': category_name, 'it': c_it, 'data': data_vis})

    model_counter[category_id] += 1

In [ ]:
# model = config.get_model(cfg, device=device, dataset=train_dataset)
ref_encoder=LocalPoolPointnet(c_dim=32, dim=3, hidden_dim=32, scatter_type='max', 
unet=False, unet_kwargs=None, unet3d=True, unet3d_kwargs={"num_levels":3,"f_maps":32,"in_channels":32,"out_channels":32}, plane_resolution=None,grid_resolution=32, plane_type='grid', padding=0.1, n_blocks=5)
my_encoder=local_pool_pn(c_dim=32,mid_dim=32,scatter_type="max",unet3d=True,unet3d_kwargs={"num_levels":3,"f_maps":32,"in_channels":32,"out_channels":32},
                         grid_resolution=32,padding=0.1,n_blocks=5)

ref_decoder=LocalDecoder(dim=3, c_dim=32,hidden_size=32, n_blocks=5, leaky=False, sample_mode='bilinear', padding=0.1)
my_decoder=local_decoder(point_dim=3,feat_dim=32,leaky=False,sample_mode="bilinear",padding=0.1,mid_dim=32,n_blocks=5)

my_model=conv_occ_net(encoder=my_encoder,decoder=my_decoder,device=device)
ref_model=conv_occ_net(encoder=ref_encoder,decoder=ref_decoder,device=device)

In [ ]:
generator = generation.Generator3D(my_model,device=device,threshold=cfg['test']['threshold'],resolution0=cfg['generation']['resolution_0'],
upsampling_steps=cfg['generation']['upsampling_steps'],sample=cfg['generation']['use_sampling'],
refinement_step=cfg['generation']['refinement_step'],simplify_nfaces=cfg['generation']['simplify_nfaces'],
input_type = cfg['data']['input_type'],padding=cfg['data']['padding'],vol_info = None,vol_bound = None)

In [ ]:
vis_dir = os.path.join(out_dir, 'vis')

my_opt=optim.Adam(my_model.parameters(), lr=1e-4)
ref_opt=optim.Adam(ref_model.parameters(), lr=1e-4)

my_trainer=Trainer(model=my_model,optimizer=my_opt,device=device,input_type="pointcloud",vis_dir=vis_dir,threshold=0.5,eval_sample=False)
ref_trainer=Trainer(model=ref_model,optimizer=ref_opt,device=device,input_type="pointcloud",vis_dir=vis_dir,threshold=0.5,eval_sample=False)

In [ ]:
checkpoint_io = CheckpointIO(out_dir, model=my_model, optimizer=my_opt)
try:
    load_dict = checkpoint_io.load('model.pt')
except FileExistsError:
    load_dict = dict()
epoch_it = load_dict.get('epoch_it', 0)
it = load_dict.get('it', 0)
metric_val_best = load_dict.get(
    'loss_val_best', -model_selection_sign * np.inf)

if metric_val_best == np.inf or metric_val_best == -np.inf:
    metric_val_best = -model_selection_sign * np.inf
print('Current best validation metric (%s): %.8f'
      % (model_selection_metric, metric_val_best))
logger = SummaryWriter(os.path.join(out_dir, 'logs'))

In [ ]:
print_every = cfg['training']['print_every']
checkpoint_every = cfg['training']['checkpoint_every']
validate_every = cfg['training']['validate_every']
visualize_every = cfg['training']['visualize_every']

# Print model
nparameters = sum(p.numel() for p in my_model.parameters())
print('Total number of parameters: %d' % nparameters)

print('output path: ', cfg['training']['out_dir'])

In [ ]:
t0 = time.time()
while True:
    epoch_it += 1

    for batch in train_loader:
        it += 1
        loss = my_trainer.train_step(batch)
        logger.add_scalar('train/loss', loss, it)

        # Print output
        if print_every > 0 and (it % print_every) == 0:
            t = datetime.datetime.now()
            print('[Epoch %02d] it=%03d, loss=%.4f, time: %.2fs, %02d:%02d'
                     % (epoch_it, it, loss, time.time() - t0, t.hour, t.minute))

        # Visualize output
        if visualize_every > 0 and (it % visualize_every) == 0:
            print('Visualizing')
            for data_vis in data_vis_list:
                if cfg['generation']['sliding_window']:
                    out = generator.generate_mesh_sliding(data_vis['data'])    
                else:
                    out = generator.generate_mesh(data_vis['data'])
                # Get statistics
                try:
                    mesh, stats_dict = out
                except TypeError:
                    mesh, stats_dict = out, {}

                mesh.export(os.path.join(out_dir, 'vis', '{}_{}_{}.off'.format(it, data_vis['category'], data_vis['it'])))
        if (checkpoint_every > 0 and (it % checkpoint_every) == 0):
            print('Saving checkpoint')
            checkpoint_io.save('model.pt', epoch_it=epoch_it, it=it,
                               loss_val_best=metric_val_best)

        # Backup if necessary
        if (backup_every > 0 and (it % backup_every) == 0):
            print('Backup checkpoint')
            checkpoint_io.save('model_%d.pt' % it, epoch_it=epoch_it, it=it,
                               loss_val_best=metric_val_best)
        # Run validation
        if validate_every > 0 and (it % validate_every) == 0:
            eval_dict = trainer.evaluate(val_loader)
            metric_val = eval_dict[model_selection_metric]
            print('Validation metric (%s): %.4f'
                  % (model_selection_metric, metric_val))

            for k, v in eval_dict.items():
                logger.add_scalar('val/%s' % k, v, it)

            if model_selection_sign * (metric_val - metric_val_best) > 0:
                metric_val_best = metric_val
                print('New best model (loss %.4f)' % metric_val_best)
                checkpoint_io.save('model_best.pt', epoch_it=epoch_it, it=it,
                                   loss_val_best=metric_val_best)

        # Exit if necessary
        if exit_after > 0 and (time.time() - t0) >= exit_after:
            print('Time limit reached. Exiting.')
            checkpoint_io.save('model.pt', epoch_it=epoch_it, it=it,
                               loss_val_best=metric_val_best)
            exit(3)

In [ ]:
!pip install plyfile

## VISUALIZATION

In [ ]:
import torch
import os
import shutil
import argparse
from tqdm import tqdm
import time
from collections import defaultdict
import pandas as pd
from checkpoints import CheckpointIO
from src.utils.io import export_pointcloud
from src.utils.visualize import visualize_data
from src.utils.voxels import VoxelGrid


import torch
import torch.optim as optim
#from tensorboardX import SummaryWriter
import numpy as np
import os
import yaml
import argparse
import time, datetime
import matplotlib; matplotlib.use('Agg')
from collections import defaultdict
import shutil


import torch.optim as optim
from trainer import Trainer
from trainer import conv_occ_net
from encoder import local_pool_pn
from ref_encoder import LocalPoolPointnet
from decoder import local_decoder
from ref_decoder import LocalDecoder

from get_dataset import get_data_fields
from get_dataset import Shapes3dDataset
from get_dataset import get_inputs_field
from get_dataset import collate_remove_none
import generation
from checkpoints import CheckpointIO

In [ ]:
import numpy as np
import math
import random
import os
import torch
import scipy.spatial.distance
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import plotly.graph_objects as go
import plotly.express as px

In [ ]:
def read_off(file):
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return verts, faces

In [ ]:
filename="60000_airplane_0.off"
with open(filename, 'r') as f:
  verts, faces = read_off(f)

In [ ]:
x,y,z = np.array(verts).T
i,j,k = np.array(faces).T

In [ ]:
def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1, 1., 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=ye, y=xe, z=ze))))))
    fig = go.Figure(data=data,
                    layout=go.Layout(
                        updatemenus=[dict(type='buttons',showactive=False,y=1,x=1,xanchor='left',yanchor='bottom',pad=dict(t=45, r=10),
                        buttons=[dict(label='Play',method='animate',args=[None, dict(frame=dict(duration=50, redraw=True),transition=dict(duration=0),
                        fromcurrent=True,mode='immediate')])])]),frames=frames)
    return fig

In [ ]:
visualize_rotate([go.Mesh3d(x=x, y=y, z=z, color='lightpink', opacity=0.50, i=i,j=j,k=k)]).show()

In [ ]:
import plotly.express as px
fig.write_html(filename.split(".")[0]+".html")

In [ ]:
filename.split(".")[0]+".html"

'60000_airplane_0.html'